In [20]:
# coding:utf8
import requests
import random
from lxml import etree
import time
import pandas as pd
# 提供ua信息的的包
# from uainfo import ua_list
import pymysql


class MeituanSpider(object):
    def __init__(self):
        self.url = 'https://minsu.dianping.com/beijing/pn{}/?dateBegin=20221101&dateEnd=20221102'
        # 计数，请求一个页面的次数，初始值为1
        self.mysql = pymysql.connect(host='localhost', database='test', port=3306, user='root',
                                         password='root')
        self.cur = self.mysql.cursor()
        self.blog = 1
        # 随机取一个UA
   
        # 发送请求
    def get_html(self, url):
        # 在超时间内，对于失败页面尝试请求三次
        if self.blog <= 3:
            try:
                data = {'dateBegin': '20221101',
                        'dateEnd': '20221102'
                }
                headers = {'Cookie': '_lxsdk_cuid=18a8d26aab5c8-05eb1d1a0c5011-19525634-13c680-18a8d26aab5c6; WEBDFPID=653629w0044851z107vuy568v7431w7y81z1w73y3v897958u0yy856y-2009945240344-1694585237637IUEEIMG75613c134b6a252faa6802015be905516892; _hc.v=be61828e-7424-ecd6-0600-adbd08fdd53a.1694585248; _gid=GA1.2.315039336.1709511959; zgwww=ec96e200-d9c5-11ee-b1e2-554576b05e95; uuid=737AD5AA843EC6B2CF5BE92E56A3CF1636C616692CDDB0482F15276FE8D97C62; iuuid=737AD5AA843EC6B2CF5BE92E56A3CF1636C616692CDDB0482F15276FE8D97C62; _lxsdk=737AD5AA843EC6B2CF5BE92E56A3CF1636C616692CDDB0482F15276FE8D97C62; zg.userid.untrusted=465229356; token2=AgHtI4uDBpDwo45C9HKX2VwHrafhKdLxgtTHq7uvDbAA8nhLKhicz8EShLxHZJ2PuGxz9P3ghsrVdAAAAABgHgAAkT4FsCnXJpfS1ZUha24p4Odi3_47frM_r8spA2mR3gcWwyE2cgxx8frQoqsdGWZ_; userid=2583015098; _ga=GA1.2.2044757989.1694585236; _lxsdk_s=18e094cc37d-28e-811-732%7C%7C65; XSRF-TOKEN=6J1cPwUh-tJr9TiF1qC_019SxF9g3V5TE8aQ; _ga_14F924BYNN=GS1.1.1709553024.3.1.1709553637.60.0.0',
                            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3928.4 Safari/537.36',}
                res = requests.get(url=url, headers=headers,data=data, timeout=3)
                html = res.text
                return html
            except Exception as e:
                print(e)
                self.blog += 1
                self.get_html(url)

        # 解析提取数据
    def parse_html(self, url):
        html = self.get_html(url)
        if html:
            time.sleep(1)
            p = etree.HTML(html)
            # 基准xpath表达式-30个房源节点对象列表
            h_list = p.xpath('//div[@class="r-card-list v-stretch h-stretch"]/div[@class="r-card-list__item shrink-in-sm"]')
            # 所有列表节点对象
            for h in h_list:
                title1 = h.xpath('.//div[@class="product-card__title"]/text()')[0]
                address1 = h.xpath('.//div[@class="mt-2"]/text()')[0]
                url1 = 'https://minsu.dianping.com' + h.xpath('.//a[@class="product-card-container"]/@href')[0]
                configuration1 = h.xpath('.//div[@class="product-card__type-detail"]/div[1]/text()')[0]
                price1= h.xpath('.//span[@class ="product-card__price__latest"]/text()')[0]
                collections1 = h.xpath('.//span[@class="product-fav-count"]/text()')[0]
#                 result = requests.get(url=url1, headers=self.get_header())
#                 tree = etree.HTML(result.text)
#                 print(tree)
                # time.sleep(1.0)
                # price = tree.xpath("//div[@class='price ']/span/text()")[0]
                data = {
                    'title': title1,
                    'urls': url1,
                    'address': address1,
                    'configuration': configuration1,
                    'collections': collections1,
                    'price': price1,
                    'unit': '元'
                    }

                print(data)
                df = pd.DataFrame(data,index=range(0, 3))
                df.to_csv('house.csv', index=False, encoding='utf-8')
#                 self.save_mysql(data)

    def save_mysql(self, data):
        # str_sql = "insert into ftx values(0, '{}', '{}');".format(data['first_category'],data['second_category'])
        str_sql = "insert into meituan values(0, '{}', '{}', '{}', '{}', '{}', '{}', '{}');".format(
                data['title'],  data['urls'],data['address'], data['configuration'], data['collections'],
                data['price'], data['unit'])

        self.cur.execute(str_sql)
        self.mysql.commit()

    def __del__(self):
        self.cur.close()
        self.mysql.close()


        # 入口函数
    def run(self):
        try:
            for i in range(1,4):
                url = self.url.format(i)
                print(i)

                self.parse_html(url)
                time.sleep(random.randint(3, 5))
                # 每次抓取一页要初始化一次self.blog
                self.blog = 1
        except Exception as e:
            print('发生错误', e)


if __name__ == '__main__':
    spider = MeituanSpider()
    spider.run()

1
{'title': '【乐朵·短租】温馨loft/良乡大学城/近地铁/理工大学/大床情侣房/短租', 'urls': 'https://minsu.dianping.com/housing/1030591708/', 'address': '良乡大学城', 'configuration': '整套房源1室 · 1床 · 可住2人', 'collections': '2', 'price': '298', 'unit': '元'}
{'title': '【兴宿-独立套房】近国贸/SKP/三里屯/*/欢乐谷/百子湾，距离地铁7号线仅700米', 'urls': 'https://minsu.dianping.com/housing/840394414/', 'address': '国贸CBD/四惠', 'configuration': '整套房源1室 · 1床 · 可住2~3人', 'collections': '108', 'price': '598', 'unit': '元'}
{'title': '【留念·投影轻奢】恒大精装loft私人影院 大一居公寓（干湿分离）直达西单动物园海洋馆北京南站大兴机场**南锣', 'urls': 'https://minsu.dianping.com/housing/943440656/', 'address': '北京大兴国际机场', 'configuration': '整套房源1室 · 1床 · 可住2人', 'collections': '47', 'price': '388', 'unit': '元'}
{'title': '临近北京南站、大红门、公益西桥、地铁10号线角门东、可短租可长租、王府井奥莱、天坛yi院、', 'urls': 'https://minsu.dianping.com/housing/1028858918/', 'address': '永定门/大红门/南苑/北京南站地区', 'configuration': '整套房源1室 · 1床 · 可住2~4人', 'collections': '35', 'price': '259', 'unit': '元'}
{'title': '【自在·绚丽霓虹 】复式loft 一居电视近地铁/奥莱/良乡大学城公园/时代广场/', 'urls': 'https://mi